In [53]:
from datetime import datetime, timedelta
import pickle
from time import sleep

from bs4 import BeautifulSoup
from requests_html import HTMLSession
import yfinance as yf

In [2]:
EXCHANGE_PATHS = {
    'nasdaq': 'nasdaq',
    'nyse': 'newyorkstockexchange',
    'amex': 'americanstockexchange'}
base_url = 'https://www.advfn.com'
session = HTMLSession()

In [3]:
def get_soup(url):
    try:
        res = session.get(url)
        html = res.html.html
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    except BaseException as e:
        print(f'Problem retrieving page: {url}\n{e}')

In [4]:
def get_table(soup):
    try:
        table = soup.find('table', class_='market')
        return table
    except BaseExcetpion as e:
        print('Unable to obtain table')
        print(e)

In [5]:
def get_rows(table):
    try:
        rows = table.find_all('tr')
        rows = [
            row for row in rows if 'class' in row.attrs 
            and row.attrs['class'][0].startswith('ts')]
        return rows
    except BaseException as e:
        print('Unable to extract rows')
        print(e)

In [6]:
def is_valid(symbol):
    MAXLEN = 4
    return len(symbol) <= MAXLEN and symbol.isalpha()

In [7]:
def get_symbol(row):
    SYMBOL_COL_IDX = 1
    try:
        symbol = row.find_all('a')[SYMBOL_COL_IDX].text
        if is_valid(symbol):
            return symbol
    except BaseException as e:
        print('Unable to get symbol from row:', row)
        print(e)

In [8]:
def extract_symbols(soup):
    symbols = []
    table = get_table(soup)
    rows = get_rows(table)
    for row in rows:
        symbol = get_symbol(row)
        if symbol is not None:
            symbols.append(symbol)
    return symbols

In [9]:
def extract_all_symbols():
    all_symbols = []
    for exchange, path in EXCHANGE_PATHS.items():
        print(f'Beggining {exchange}\nLetter:')
        for letter in list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
            print(letter, end='')
            url = f'{base_url}/{exchange}/{path}.asp?companies={letter}'
            soup = get_soup(url)
            symbols = extract_symbols(soup)
            all_symbols += symbols
        print()
    return sorted(all_symbols)

In [10]:
all_symbols = extract_all_symbols()

Beggining nasdaq
Letter:
ABCDEFGHIJKLMNOPQRSTUVWXYZ
Beggining nyse
Letter:
ABCDEFGHIJKLMNOPQRSTUVWXYZ
Beggining amex
Letter:
ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [11]:
all_symbols = sorted(list(set(all_symbols)))

In [12]:
all_symbols[:10]

['A', 'AA', 'AAAP', 'AABA', 'AABC', 'AAC', 'AACC', 'AACE', 'AACI', 'AACQ']

In [13]:
all_symbols[-10:]

['ZWRK', 'ZWS', 'ZY', 'ZYME', 'ZYNE', 'ZYXI', 'ZZK', 'ZZO', 'ZZX', 'ZZZ']

In [14]:
with open('../data/all_symbols.pkl', 'wb') as f:
    pickle.dump(all_symbols, f)

In [34]:
n = len(all_symbols)
n

11674

In [54]:
END = datetime.now()
START = END - timedelta(7)
test = yf.download(['AAPL', 'TSLA', 'ZQZZ'], start=str(START.date()), end=str(END.date()))['Adj Close']
test

[*********************100%***********************]  3 of 3 completed

1 Failed download:
- ZQZZ: No timezone found, symbol may be delisted


,AAPL,TSLA,ZQZZ
Date,,,
2024-06-24 00:00:00,208.139999,182.580002,NaN
2024-06-25 00:00:00,209.070007,187.350006,NaN
2024-06-26 00:00:00,213.250000,196.369995,NaN
2024-06-27 00:00:00,214.100006,197.419998,NaN
2024-06-28 00:00:00,210.619995,197.880005,NaN


In [47]:
test2 = test.loc[:, test.iloc[-1, :].notnull()]
test2

,AAPL,TSLA
Date,,
2024-06-24 00:00:00,208.139999,182.580002
2024-06-25 00:00:00,209.070007,187.350006
2024-06-26 00:00:00,213.250000,196.369995
2024-06-27 00:00:00,214.100006,197.419998
2024-06-28 00:00:00,210.619995,197.880005


In [48]:
test2.columns

Index(['AAPL', 'TSLA'], dtype='object')

In [56]:
start = str(START.date())
end = str(END.date())
start, end

('2024-06-23', '2024-06-30')

In [58]:
i = 0
active = []
failed = []

while i <= n:
    first = i
    last = min(i + 100, n)
    batch = all_symbols[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    df = df.loc[:, df.iloc[-1, :].notnull()]
    active += list(df.columns)
    fails = [sym for sym in batch if sym not in df.columns]
    failed += fails
    sleep(3)
    i += 100

[*********************100%***********************]  100 of 100 completed

56 Failed downloads:
- ACAX: No data found, symbol may be delisted
- AAWW: No data found, symbol may be delisted
- ABTL: No data found for this date range, symbol may be delisted
- AANB: No data found for this date range, symbol may be delisted
- AACC: No data found for this date range, symbol may be delisted
- ABBI: No data found for this date range, symbol may be delisted
- ABLX: No data found for this date range, symbol may be delisted
- ABCW: No data found for this date range, symbol may be delisted
- AAPC: No data found for this date range, symbol may be delisted
- ABST: No data found, symbol may be delisted
- ACAP: No data found for this date range, symbol may be delisted
- AATI: No data found for this date range, symbol may be delisted
- ACC: No timezone found, symbol may be delisted
- AAC: No data found, symbol may be delisted
- AAII: No data found for this date range, symbol may be delisted
- ACCL: No da

[*********************100%***********************]  100 of 100 completed

46 Failed downloads:
- AKRX: No timezone found, symbol may be delisted
- AIW: No timezone found, symbol may be delisted
- AJRD: No data found, symbol may be delisted
- AHAC: No timezone found, symbol may be delisted
- AIMT: No timezone found, symbol may be delisted
- AGN: No timezone found, symbol may be delisted
- AIKI: No data found, symbol may be delisted
- AHRN: No data found, symbol may be delisted
- AJXA: No data found for this date range, symbol may be delisted
- AKCA: No timezone found, symbol may be delisted
- AIC: No data found, symbol may be delisted
- AGU: No data found for this date range, symbol may be delisted
- AGLE: No data found, symbol may be delisted
- AKLM: No data found for this date range, symbol may be delisted
- AHC: No timezone found, symbol may be delisted
- AIXD: No data found for this date range, symbol may be delisted
- AIPT: No timezone found, symbol may be delisted
- AGTC: No data 

[*********************100%***********************]  100 of 100 completed

53 Failed downloads:
- ARYB: No timezone found, symbol may be delisted
- ARWA: No data found for this date range, symbol may be delisted
- AREX: No timezone found, symbol may be delisted
- ARRO: No data found for this date range, symbol may be delisted
- ARYE: No data found, symbol may be delisted
- ARVL: No data found, symbol may be delisted
- ARTG: No data found for this date range, symbol may be delisted
- ARAV: No data found, symbol may be delisted
- ASAQ: No timezone found, symbol may be delisted
- ARCE: No data found, symbol may be delisted
- ARRW: No data found, symbol may be delisted
- ARCW: No data found, symbol may be delisted
- ARG: No data found for this date range, symbol may be delisted
- ARIZ: No data found, symbol may be delisted
- ARCK: No data found, symbol may be delisted
- ARGN: No data found for this date range, symbol may be delisted
- ARNA: No timezone found, symbol may be delisted
- ARBR: 

[*********************100%***********************]  100 of 100 completed

38 Failed downloads:
- BBBB: No data found for this date range, symbol may be delisted
- AXCA: No data found for this date range, symbol may be delisted
- AXLA: Period 'max' is invalid, must be one of ['1d', '5d']
- BBBY: No data found, symbol may be delisted
- AXGT: No timezone found, symbol may be delisted
- BABS: No data found for this date range, symbol may be delisted
- AXH: No data found, symbol may be delisted
- BAMH: No data found, symbol may be delisted
- BALB: No data found for this date range, symbol may be delisted
- BARE: No timezone found, symbol may be delisted
- AYR: No timezone found, symbol may be delisted
- BBCO: Period 'max' is invalid, must be one of ['1d', '5d']
- AYX: No data found, symbol may be delisted
- AXYS: No data found for this date range, symbol may be delisted
- AZRX: No timezone found, symbol may be delisted
- AYT: No data found for this date range, symbol may be delisted
- BASI:

[*********************100%***********************]  100 of 100 completed

46 Failed downloads:
- BNSO: No data found, symbol may be delisted
- BPRG: No data found for this date range, symbol may be delisted
- BMET: No data found for this date range, symbol may be delisted
- BPYU: No timezone found, symbol may be delisted
- BPFH: No timezone found, symbol may be delisted
- BOXC: No data found for this date range, symbol may be delisted
- BNCL: No timezone found, symbol may be delisted
- BOBE: No data found for this date range, symbol may be delisted
- BOSA: No data found for this date range, symbol may be delisted
- BOAC: No data found, symbol may be delisted
- BOBJ: No data found for this date range, symbol may be delisted
- BOMN: No timezone found, symbol may be delisted
- BPR: No timezone found, symbol may be delisted
- BNHN: No data found for this date range, symbol may be delisted
- BOXD: No data found, symbol may be delisted
- BNFT: No data found, symbol may be delisted
- BOAS: No

...

[*********************100%***********************]  100 of 100 completed

54 Failed downloads:
- TCNO: No data found for this date range, symbol may be delisted
- SZC: No timezone found, symbol may be delisted
- TARO: No data found for this date range, symbol may be delisted
- SYXI: No data found for this date range, symbol may be delisted
- TALL: No data found for this date range, symbol may be delisted
- TCRD: No timezone found, symbol may be delisted
- TCO: No timezone found, symbol may be delisted
- TALX: No data found for this date range, symbol may be delisted
- TCRR: No data found, symbol may be delisted
- TBDI: No data found for this date range, symbol may be delisted
- TBSI: No data found for this date range, symbol may be delisted
- TCF: No timezone found, symbol may be delisted
- TCNJ: No data found for this date range, symbol may be delisted
- TCDA: No data found, symbol may be delisted
- TBUS: No data found for this date range, symbol may be delisted
- SYUT: No data found 

[*********************100%***********************]  100 of 100 completed

56 Failed downloads:
- TRCH: No timezone found, symbol may be delisted
- TMWE: No timezone found, symbol may be delisted
- TMSR: No timezone found, symbol may be delisted
- TNAV: No timezone found, symbol may be delisted
- TPIV: No data found for this date range, symbol may be delisted
- TRBC: No data found for this date range, symbol may be delisted
- TRAD: No data found for this date range, symbol may be delisted
- TPTH: No data found for this date range, symbol may be delisted
- TPS: No data found for this date range, symbol may be delisted
- TRAQ: No data found, symbol may be delisted
- TOPP: No data found for this date range, symbol may be delisted
- TRCR: No data found for this date range, symbol may be delisted
- TRCB: No timezone found, symbol may be delisted
- TMTS: No timezone found, symbol may be delisted
- TNCC: No data found for this date range, symbol may be delisted
- TNTX: No data found for this d

[*********************100%***********************]  100 of 100 completed

48 Failed downloads:
- UNXL: No data found for this date range, symbol may be delisted
- UNCL: No timezone found, symbol may be delisted
- UBM: No data found for this date range, symbol may be delisted
- UNIZ: No data found for this date range, symbol may be delisted
- UGCE: No data found for this date range, symbol may be delisted
- UBN: No data found for this date range, symbol may be delisted
- UCNN: No data found for this date range, symbol may be delisted
- UFAB: No data found, symbol may be delisted
- UNCA: No data found for this date range, symbol may be delisted
- UBET: No data found for this date range, symbol may be delisted
- UBA: No data found, symbol may be delisted
- UNUM: No timezone found, symbol may be delisted
- UNBJ: No data found for this date range, symbol may be delisted
- UBRT: No data found for this date range, symbol may be delisted
- UBSH: No timezone found, symbol may be delisted
- UBNK

[*********************100%***********************]  100 of 100 completed

47 Failed downloads:
- VSCE: No data found for this date range, symbol may be delisted
- VWKS: No data found for this date range, symbol may be delisted
- VTIQ: No data found, symbol may be delisted
- VRLK: No data found for this date range, symbol may be delisted
- VSEA: No data found for this date range, symbol may be delisted
- VSNT: No data found for this date range, symbol may be delisted
- VSLR: No timezone found, symbol may be delisted
- VWPT: No data found for this date range, symbol may be delisted
- VRTU: No timezone found, symbol may be delisted
- VWTR: No data found for this date range, symbol may be delisted
- VTA: No timezone found, symbol may be delisted
- VRAZ: No timezone found, symbol may be delisted
- VVUS: No timezone found, symbol may be delisted
- VSM: No timezone found, symbol may be delisted
- VRNM: No data found for this date range, symbol may be delisted
- VRNG: No data found for this da

[*********************100%***********************]  100 of 100 completed

56 Failed downloads:
- WPG: No timezone found, symbol may be delisted
- WTRH: No data found, symbol may be delisted
- WSII: No data found for this date range, symbol may be delisted
- WSBA: No data found for this date range, symbol may be delisted
- WTR: No timezone found, symbol may be delisted
- WTBK: No data found for this date range, symbol may be delisted
- WSSI: No data found for this date range, symbol may be delisted
- WXCO: No timezone found, symbol may be delisted
- WTRU: No timezone found, symbol may be delisted
- WR: No data found for this date range, symbol may be delisted
- WQGA: No data found, symbol may be delisted
- WWWB: No data found for this date range, symbol may be delisted
- WTCO: No data found for this date range, symbol may be delisted
- WRLS: No timezone found, symbol may be delisted
- WRAC: No data found, symbol may be delisted
- WSPI: No data found for this date range, symbol may be de

### 2nd Attempt

In [60]:
len(active)

5831

In [59]:
n = len(failed)
n

5769

In [65]:
failed_again = []
i = 0
while i <= n:
    first = i
    last = min(i + 100, n)
    batch = failed[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    print(df.shape)
    if len(df):
        df = df.loc[:, df.iloc[-1, :].notnull()]
        active += list(df.columns)
        fails = [sym for sym in batch if sym not in df.columns]
        failed_again += fails
    else:
        failed_again += batch
    sleep(3)
    i += 100

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- ABTL: No data found for this date range, symbol may be delisted
- AAWW: No data found, symbol may be delisted
- ACAX: No data found, symbol may be delisted
- ACOM: No data found for this date range, symbol may be delisted
- AAPC: No data found for this date range, symbol may be delisted
- AACC: No data found for this date range, symbol may be delisted
- ABBI: No data found for this date range, symbol may be delisted
- AANB: No data found for this date range, symbol may be delisted
- ADBL: No data found for this date range, symbol may be delisted
- ACHN: No data found, symbol may be delisted
- ACER: No data found, symbol may be delisted
- ACAP: No data found for this date range, symbol may be delisted
- ABCW: No data found for this date range, symbol may be delisted
- ACTU: No data found for this date range, symbol may be delisted
- ACFC: No data found for this date range, symbol may be del

(4, 100)
[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- ALJJ: No data found, symbol may be delisted
- ANPI: No data found for this date range, symbol may be delisted
- AMEV: No data found for this date range, symbol may be delisted
- AMCI: No data found, symbol may be delisted
- ALTE: No data found for this date range, symbol may be delisted
- ALGX: No data found for this date range, symbol may be delisted
- ANSR: No data found for this date range, symbol may be delisted
- ALVU: Period 'max' is invalid, must be one of ['1d', '5d']
- AMAG: No timezone found, symbol may be delisted
- ANAT: No timezone found, symbol may be delisted
- AMRH: No timezone found, symbol may be delisted
- ALPX: Period 'max' is invalid, must be one of ['1d', '5d']
- ANCC: No data found for this date range, symbol may be delisted
- AMQ: Period 'max' is invalid, must be one of ['1d', '5d']
- ALDN: No data found for this date range, symbol may be delisted
- ANGN: N

(0, 100)
[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- ATA: No data found, symbol may be delisted
- AVG: No data found for this date range, symbol may be delisted
- AUO: No timezone found, symbol may be delisted
- AVNX: No data found for this date range, symbol may be delisted
- ATVC: No timezone found, symbol may be delisted
- AVEO: No data found, symbol may be delisted
- AVNU: No data found for this date range, symbol may be delisted
- AXCA: No data found for this date range, symbol may be delisted
- ATTO: No data found, symbol may be delisted
- ATPG: No data found for this date range, symbol may be delisted
- ATSI: No data found for this date range, symbol may be delisted
- AUXG: No timezone found, symbol may be delisted
- AXLA: Period 'max' is invalid, must be one of ['1d', '5d']
- AVNC: No data found for this date range, symbol may be delisted
- AXGT: No timezone found, symbol may be delisted
- ASHW: No data found for this date r

(0, 100)
[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- BLUD: No data found for this date range, symbol may be delisted
- BMET: No data found for this date range, symbol may be delisted
- BLTS: No data found, symbol may be delisted
- BKBK: No data found for this date range, symbol may be delisted
- BNSO: No data found, symbol may be delisted
- BLSC: No data found for this date range, symbol may be delisted
- BIGD: No data found for this date range, symbol may be delisted
- BPRG: No data found for this date range, symbol may be delisted
- BIOS: No data found, symbol may be delisted
- BPYU: No timezone found, symbol may be delisted
- BLL: No timezone found, symbol may be delisted
- BPFH: No timezone found, symbol may be delisted
- BOXC: No data found for this date range, symbol may be delisted
- BLMT: No timezone found, symbol may be delisted
- BKST: No data found for this date range, symbol may be delisted
- BIVV: No data found for this

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- CBPX: No timezone found, symbol may be delisted
- CCMP: No timezone found, symbol may be delisted
- CDAK: Period 'max' is invalid, must be one of ['1d', '5d']
- CASB: No data found for this date range, symbol may be delisted
- CCE: No data found for this date range, symbol may be delisted
- CCRT: No data found for this date range, symbol may be delisted
- CBMX: No data found for this date range, symbol may be delisted
- CBHI: No data found for this date range, symbol may be delisted
- CBX: No data found, symbol may be delisted
- CAHS: No timezone found, symbol may be delisted
- CATS: No timezone found, symbol may be delisted
- CBEY: No data found for this date range, symbol may be delisted
- CARH: No timezone found, symbol may be delisted
- CBS: No timezone found, symbol may be delisted
- CASC: No data found for this date range, symbol may be delisted
- CAFN: No data found for this date ra

...

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- SSC: No data found for this date range, symbol may be delisted
- SRF: No timezone found, symbol may be delisted
- SSAA: No data found, symbol may be delisted
- STFC: No timezone found, symbol may be delisted
- SRC: No data found, symbol may be delisted
- SUPX: No data found for this date range, symbol may be delisted
- STAT: No data found for this date range, symbol may be delisted
- SUFB: No data found for this date range, symbol may be delisted
- STNV: No data found for this date range, symbol may be delisted
- STPC: No timezone found, symbol may be delisted
- SUNW: No data found, symbol may be delisted
- SRNG: No timezone found, symbol may be delisted
- SUPG: No data found for this date range, symbol may be delisted
- SSTR: No data found for this date range, symbol may be delisted
- SRRR: No data found for this date range, symbol may be delisted
- SVAC: No timezone found, symbol may be 

(0, 100)
[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- TINV: No data found, symbol may be delisted
- TEC: No data found for this date range, symbol may be delisted
- THRX: No data found, symbol may be delisted
- THOR: No timezone found, symbol may be delisted
- TMWE: No timezone found, symbol may be delisted
- TELM: No data found for this date range, symbol may be delisted
- TLRD: No timezone found, symbol may be delisted
- TFM: No data found for this date range, symbol may be delisted
- TMSR: No timezone found, symbol may be delisted
- TMH: No data found for this date range, symbol may be delisted
- TIER: No timezone found, symbol may be delisted
- TISA: No timezone found, symbol may be delisted
- TLVT: No data found for this date range, symbol may be delisted
- TELN: No data found for this date range, symbol may be delisted
- THMA: No timezone found, symbol may be delisted
- TGE: No timezone found, symbol may be delisted
- TMTS: No 

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- TUNE: No data found for this date range, symbol may be delisted
- TUGC: No timezone found, symbol may be delisted
- UNCL: No timezone found, symbol may be delisted
- TXTR: No data found for this date range, symbol may be delisted
- TWTR: No data found, symbol may be delisted
- UAPH: No data found for this date range, symbol may be delisted
- TVAC: No timezone found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
- UNIZ: No data found for this date range, symbol may be delisted
- UBM: No data found for this date range, symbol may be delisted
- TWCT: No timezone found, symbol may be delisted
- TYHT: No timezone found, symbol may be delisted
- UGCE: No data found for this date range, symbol may be delisted
- UCNN: No data found for this date range, symbol may be delisted
- TUEM: No data found, symbol may be delisted
- UBN: No data found for this date r

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- VSCE: No data found for this date range, symbol may be delisted
- VSLR: No timezone found, symbol may be delisted
- VSNT: No data found for this date range, symbol may be delisted
- VTIQ: No data found, symbol may be delisted
- VSEA: No data found for this date range, symbol may be delisted
- VRLK: No data found for this date range, symbol may be delisted
- VNTV: No data found for this date range, symbol may be delisted
- VWKS: No data found for this date range, symbol may be delisted
- VWPT: No data found for this date range, symbol may be delisted
- VLML: No data found for this date range, symbol may be delisted
- VRTU: No timezone found, symbol may be delisted
- VTA: No timezone found, symbol may be delisted
- VWTR: No data found for this date range, symbol may be delisted
- VRAZ: No timezone found, symbol may be delisted
- VVUS: No timezone found, symbol may be delisted
- VNE: No timez

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- XE: No data found for this date range, symbol may be delisted
- WSBA: No data found for this date range, symbol may be delisted
- WTRH: No data found, symbol may be delisted
- XCYT: No data found for this date range, symbol may be delisted
- WSII: No data found for this date range, symbol may be delisted
- XETA: No data found for this date range, symbol may be delisted
- XF: Period 'max' is invalid, must be one of ['1d', '5d']
- XFML: No timezone found, symbol may be delisted
- WTR: No timezone found, symbol may be delisted
- XENT: No timezone found, symbol may be delisted
- XLYS: Period 'max' is invalid, must be one of ['1d', '5d']
- WTBK: No data found for this date range, symbol may be delisted
- XMLG: No data found for this date range, symbol may be delisted
- WTRU: No timezone found, symbol may be delisted
- WR: No data found for this date range, symbol may be delisted
- WSSI: No data

### 3rd Attempt

In [66]:
len(active)

5837

In [67]:
failed = failed_again[:]
n = len(failed)
n

5694

In [68]:
failed_again = []
i = 0
while i <= n :
    first = i
    last = min(i + 100, n)
    batch = failed[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    print(df.shape)
    if len(df):
        df = df.loc[:, df.iloc[-1, :].notnull()]
        active += list(df.columns)
        fails = [sym for sym in batch if sym not in df.columns]
        failed_again += fails
    else:
        failed_again += batch
    sleep(3)
    i += 100

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- AACC: No data found for this date range, symbol may be delisted
- AAWW: No data found, symbol may be delisted
- ACOM: No data found for this date range, symbol may be delisted
- ABTL: No data found for this date range, symbol may be delisted
- AANB: No data found for this date range, symbol may be delisted
- AAPC: No data found for this date range, symbol may be delisted
- ABBI: No data found for this date range, symbol may be delisted
- ACAX: No data found, symbol may be delisted
- ADBL: No data found for this date range, symbol may be delisted
- ACHN: No data found, symbol may be delisted
- ACFC: No data found for this date range, symbol may be delisted
- ACER: No data found, symbol may be delisted
- ABCW: No data found for this date range, symbol may be delisted
- ACXM: No data found for this date range, symbol may be delisted
- ACAP: No data found for this date range, symbol may be del

(0, 100)
[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- AMEV: No data found for this date range, symbol may be delisted
- ANPI: No data found for this date range, symbol may be delisted
- ALJJ: No data found, symbol may be delisted
- AMCI: No data found, symbol may be delisted
- ALVU: Period 'max' is invalid, must be one of ['1d', '5d']
- ANSR: No data found for this date range, symbol may be delisted
- ALTE: No data found for this date range, symbol may be delisted
- ALGX: No data found for this date range, symbol may be delisted
- AMAG: No timezone found, symbol may be delisted
- AMRH: No timezone found, symbol may be delisted
- ANCC: No data found for this date range, symbol may be delisted
- ANAT: No timezone found, symbol may be delisted
- ANGN: No data found, symbol may be delisted
- ALDN: No data found for this date range, symbol may be delisted
- AMQ: Period 'max' is invalid, must be one of ['1d', '5d']
- AMCP: No data found fo

(0, 100)
[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- ATA: No data found, symbol may be delisted
- AVNX: No data found for this date range, symbol may be delisted
- ATVC: No timezone found, symbol may be delisted
- AVEO: No data found, symbol may be delisted
- AVG: No data found for this date range, symbol may be delisted
- AXCA: No data found for this date range, symbol may be delisted
- AUO: No timezone found, symbol may be delisted
- AVNU: No data found for this date range, symbol may be delisted
- ATTO: No data found, symbol may be delisted
- ASHW: No data found for this date range, symbol may be delisted
- ATPG: No data found for this date range, symbol may be delisted
- AUXG: No timezone found, symbol may be delisted
- ATSI: No data found for this date range, symbol may be delisted
- AXLA: Period 'max' is invalid, must be one of ['1d', '5d']
- AXGT: No timezone found, symbol may be delisted
- AVNC: No data found for this date r

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- BLTS: No data found, symbol may be delisted
- BLSC: No data found for this date range, symbol may be delisted
- BPRG: No data found for this date range, symbol may be delisted
- BLUD: No data found for this date range, symbol may be delisted
- BNSO: No data found, symbol may be delisted
- BKBK: No data found for this date range, symbol may be delisted
- BMET: No data found for this date range, symbol may be delisted
- BIGD: No data found for this date range, symbol may be delisted
- BPYU: No timezone found, symbol may be delisted
- BIVV: No data found for this date range, symbol may be delisted
- BLL: No timezone found, symbol may be delisted
- BPFH: No timezone found, symbol may be delisted
- BKST: No data found for this date range, symbol may be delisted
- BIOS: No data found, symbol may be delisted
- BOXC: No data found for this date range, symbol may be delisted
- BNCL: No timezone fou

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- CBPX: No timezone found, symbol may be delisted
- CDAK: Period 'max' is invalid, must be one of ['1d', '5d']
- CCMP: No timezone found, symbol may be delisted
- CASB: No data found for this date range, symbol may be delisted
- CCRT: No data found for this date range, symbol may be delisted
- CBHI: No data found for this date range, symbol may be delisted
- CBMX: No data found for this date range, symbol may be delisted
- CCE: No data found for this date range, symbol may be delisted
- CASC: No data found for this date range, symbol may be delisted
- CBX: No data found, symbol may be delisted
- CBEY: No data found for this date range, symbol may be delisted
- CBS: No timezone found, symbol may be delisted
- CAHS: No timezone found, symbol may be delisted
- CATS: No timezone found, symbol may be delisted
- CBMB: No timezone found, symbol may be delisted
- CARH: No timezone found, symbol may 

...

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- SSC: No data found for this date range, symbol may be delisted
- STPC: No timezone found, symbol may be delisted
- STAT: No data found for this date range, symbol may be delisted
- SUPX: No data found for this date range, symbol may be delisted
- SSAA: No data found, symbol may be delisted
- STNV: No data found for this date range, symbol may be delisted
- SUFB: No data found for this date range, symbol may be delisted
- STFC: No timezone found, symbol may be delisted
- SRRR: No data found for this date range, symbol may be delisted
- SVFD: No data found, symbol may be delisted
- SUNW: No data found, symbol may be delisted
- SVLF: No data found for this date range, symbol may be delisted
- SVAC: No timezone found, symbol may be delisted
- SUPG: No data found for this date range, symbol may be delisted
- SRNG: No timezone found, symbol may be delisted
- SSTR: No data found for this date ran

(0, 100)
[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- TELM: No data found for this date range, symbol may be delisted
- TINV: No data found, symbol may be delisted
- THOR: No timezone found, symbol may be delisted
- TFG: No data found for this date range, symbol may be delisted
- TMWE: No timezone found, symbol may be delisted
- THRX: No data found, symbol may be delisted
- TLRD: No timezone found, symbol may be delisted
- TFM: No data found for this date range, symbol may be delisted
- TISA: No timezone found, symbol may be delisted
- TNAV: No timezone found, symbol may be delisted
- TMSR: No timezone found, symbol may be delisted
- TELN: No data found for this date range, symbol may be delisted
- TLVT: No data found for this date range, symbol may be delisted
- TGE: No timezone found, symbol may be delisted
- TIER: No timezone found, symbol may be delisted
- TMH: No data found for this date range, symbol may be delisted
- THMA: No 

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- UNXL: No data found for this date range, symbol may be delisted
- TVAC: No timezone found, symbol may be delisted
- TXTR: No data found for this date range, symbol may be delisted
- TUGC: No timezone found, symbol may be delisted
- UNCL: No timezone found, symbol may be delisted
- TUNE: No data found for this date range, symbol may be delisted
- TWTR: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
- UAPH: No data found for this date range, symbol may be delisted
- UBM: No data found for this date range, symbol may be delisted
- UNIZ: No data found for this date range, symbol may be delisted
- TUEM: No data found, symbol may be delisted
- UGCE: No data found for this date range, symbol may be delisted
- TYHT: No timezone found, symbol may be delisted
- TWCT: No timezone found, symbol may be delisted
- UCNN: No data found for this date 

[*********************100%***********************]  100 of 100 completed

100 Failed downloads:
- VTIQ: No data found, symbol may be delisted
- VSLR: No timezone found, symbol may be delisted
- WAAS: No timezone found, symbol may be delisted
- VSNT: No data found for this date range, symbol may be delisted
- VSCE: No data found for this date range, symbol may be delisted
- VWKS: No data found for this date range, symbol may be delisted
- WAIN: No data found for this date range, symbol may be delisted
- VNTV: No data found for this date range, symbol may be delisted
- VSEA: No data found for this date range, symbol may be delisted
- VRLK: No data found for this date range, symbol may be delisted
- VLML: No data found for this date range, symbol may be delisted
- VWPT: No data found for this date range, symbol may be delisted
- VNE: No timezone found, symbol may be delisted
- WAGI: No data found for this date range, symbol may be delisted
- VRTU: No timezone found, symbol may be delisted

(0, 100)


### 4th Attempt

In [69]:
len(active)

5837

In [70]:
failed = failed_again[:]
n = len(failed)
n

5600

In [71]:
with open('../data/all_symbols.pkl', 'wb') as f:
    pickle.dump(active, f)

In [ ]:
# Run...?
failed_again = []
i = 0
while i <= n - 100:
    first = i
    last = min(i + 100, n)
    batch = failed[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    print(df.shape)
    if len(df):
        df = df.loc[:, df.iloc[-1, :].notnull()]
        active += list(df.columns)
        fails = [sym for sym in batch if sym not in df.columns]
        failed_again += fails
    else:
        failed_again += batch
    sleep(3)
    i += 100